In [49]:
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_target = iris.target

In [50]:
iris_df

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm)
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2
...,...,...,...,...
145,6.7,3.0,5.2,2.3
146,6.3,2.5,5.0,1.9
147,6.5,3.0,5.2,2.0
148,6.2,3.4,5.4,2.3


## K 평균

In [81]:
from sklearn.cluster import KMeans
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import silhouette_score
import pandas as pd

# 데이터셋 로드
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_target = iris.target

# 데이터셋을 train과 test로 분할
X_train, X_test, y_train, y_test = train_test_split(iris_df, iris_target, test_size=0.3, random_state=42)

# 데이터 정규화
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 최적의 클러스터 수 찾기 (엘보우 방법)
inertia = []
silhouette_scores = []
K = range(2, 11)  # 클러스터의 수를 2부터 10까지 시험해본다.
for k in K:
    kmeans_model = KMeans(n_clusters=k, random_state=42)
    kmeans_model.fit(X_train_scaled)
    inertia.append(kmeans_model.inertia_)
    silhouette_scores.append(silhouette_score(X_train_scaled, kmeans_model.labels_))

# 최적의 클러스터 수를 선택하는 것은 실루엣 점수가 높고 관성이 낮은 지점을 찾는 것을 포함한다.
# 여기서는 이 단계를 단순화하고 예제의 목적상 n_clusters=3을 사용하겠다.

# KMeans 모델 생성 및 학습
kmeans = KMeans(n_clusters=3, random_state=42)
kmeans.fit(X_train_scaled)

# 테스트 데이터셋에 대한 클러스터 예측
predicted_labels = kmeans.predict(X_test_scaled)

import numpy as np

# 다시 매핑 함수를 사용하여 예측된 클러스터 레이블을 실제 레이블에 매핑
def map_clusters_to_labels(predicted_labels, true_labels):
    mapped_labels = np.zeros_like(predicted_labels)
    for i in range(kmeans.n_clusters):
        mask = (predicted_labels == i)
        mapped_labels[mask] = np.bincount(true_labels[mask]).argmax()
    return mapped_labels

mapped_labels = map_clusters_to_labels(predicted_labels, y_test)

# 매핑된 레이블로 새로운 혼동 행렬 생성
new_misclassification_table = pd.crosstab(y_test, mapped_labels, rownames=['Actual'], colnames=['Predicted'])

new_misclassification_table

# 실루엣 점수 계산
silhouette_avg = silhouette_score(X_test_scaled, predicted_labels)

misclassification_table, silhouette_avg, #inertia, #silhouette_scores

c:\Users\de31\anaconda3\envs\lp_gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\de31\anaconda3\envs\lp_gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\de31\anaconda3\envs\lp_gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
c:\Users\de31\anaconda3\envs\lp_gpu\lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will chang

(col_0   0   1   2
 row_0            
 0      19   0   0
 1       0  13   0
 2       0   0  13,
 0.431374165886554)

## 서포트 벡터 머신

In [108]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.svm import SVC
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from scipy.stats import uniform

# iris 데이터셋 로드
iris = load_iris()

# iris 데이터프레임 생성
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# iris target 값 설정
iris_target = iris.target

# train set과 test set으로 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_df, iris_target, test_size=0.3, random_state=42)

# SVM 모델 생성
svm = SVC(random_state=42)

# 랜덤 서치를 위한 매개변수 분포 설정
param_dist = {'C': uniform(loc=0.1, scale=9.9), 'kernel': ['linear', 'rbf', 'poly'], 'gamma': ['scale', 'auto']}

# 교차 검증을 포함한 랜덤 서치 수행
random_search = RandomizedSearchCV(svm, param_dist, cv=5)
random_search.fit(X_train, y_train)

# 랜덤 서치에서 최적의 모델 선택
best_svm = random_search.best_estimator_

# 테스트 데이터로 예측
predicted_labels = best_svm.predict(X_test)

# 오분류 테이블 생성
misclassification_table = pd.crosstab(y_test, predicted_labels)

# 가중치 평균 F1 스코어 계산
f1 = f1_score(y_test, predicted_labels, average='weighted')

# 정확도 계산
accuracy = accuracy_score(y_test, predicted_labels)

# 재현율 계산
recall = recall_score(y_test, predicted_labels, average='weighted')

# 정밀도 계산
precision = precision_score(y_test, predicted_labels, average='weighted')

# 오분류 테이블, F1 스코어, 정확도, 재현율, 정밀도 출력
misclassification_table, f1, accuracy, recall, precision

# 과적합 확인을 위해 훈련 데이터로 예측
train_predicted_labels = best_svm.predict(X_train)

# 훈련 데이터의 F1 스코어 계산
train_f1 = f1_score(y_train, train_predicted_labels, average='weighted')

# 훈련 데이터의 정확도 계산
train_accuracy = accuracy_score(y_train, train_predicted_labels)

# 훈련 데이터의 재현율 계산
train_recall = recall_score(y_train, train_predicted_labels, average='weighted')

# 훈련 데이터의 정밀도 계산
train_precision = precision_score(y_train, train_predicted_labels, average='weighted')

# 훈련 데이터의 F1 스코어, 정확도, 오분류 테이블, 재현율, 정밀도 출력
train_f1, train_accuracy, misclassification_table, train_recall, train_precision

(0.9809384572542468,
 0.9809523809523809,
 col_0   0   1   2
 row_0            
 0      19   0   0
 1       0  13   0
 2       0   0  13,
 0.9809523809523809,
 0.9819291819291819)

## 의사결정 나무

In [118]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# iris 데이터셋 로드
iris = load_iris()

# iris 데이터를 데이터프레임으로 변환
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# iris target 값 저장
iris_target = iris.target

# train set과 test set으로 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_df, iris_target, test_size=0.3, random_state=42)

# Decision Tree 분류기 객체 생성
dt = DecisionTreeClassifier(random_state=42)

# Random Search를 위한 매개변수 그리드 정의
param_grid = {'max_depth': [2, 3, 4, 5], 'min_samples_split': [2, 3, 4, 5]}

# Random Search 수행
random_search = RandomizedSearchCV(dt, param_distributions=param_grid, cv=5)
random_search.fit(X_train, y_train)

# Random Search로부터 최적의 분류기 객체 추출
dt_best = random_search.best_estimator_

# 테스트 데이터에 대한 예측값 생성
predicted_labels = dt_best.predict(X_test)

# 오분류 테이블 생성
misclassification_table = pd.crosstab(y_test, predicted_labels)

# 가중치 평균 F1 스코어 계산
f1 = f1_score(y_test, predicted_labels, average='weighted')

# 정확도 계산
accuracy = accuracy_score(y_test, predicted_labels)

# 재현율 계산
recall = recall_score(y_test, predicted_labels, average='weighted')

# 정밀도 계산
precision = precision_score(y_test, predicted_labels, average='weighted')

# 오분류 테이블, F1 스코어, 정확도, 재현율, 정밀도 출력
misclassification_table, f1, accuracy, recall, precision

# 과적합 확인을 위한 학습 데이터에 대한 예측값 생성
train_predicted_labels = dt_best.predict(X_train)

# 학습 데이터에 대한 가중치 평균 F1 스코어 계산
train_f1 = f1_score(y_train, train_predicted_labels, average='weighted')

# 학습 데이터에 대한 정확도 계산
train_accuracy = accuracy_score(y_train, train_predicted_labels)

# 학습 데이터에 대한 재현율 계산
train_recall = recall_score(y_train, train_predicted_labels, average='weighted')

# 학습 데이터에 대한 정밀도 계산
train_precision = precision_score(y_train, train_predicted_labels, average='weighted')

# 학습 데이터에 대한 F1 스코어, 정확도, 오분류 테이블, 재현율, 정밀도 출력
train_f1, train_accuracy, misclassification_table, train_recall, train_precision, random_search.best_params_

(0.9713815359690611,
 0.9714285714285714,
 col_0   0   1   2
 row_0            
 0      19   0   0
 1       0  13   0
 2       0   0  13,
 0.9714285714285714,
 0.9735714285714285,
 {'min_samples_split': 4, 'max_depth': 4})

In [122]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score, accuracy_score, recall_score, precision_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# iris 데이터셋 로드
iris = load_iris()

# iris 데이터를 데이터프레임으로 변환
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# iris target 값 저장
iris_target = iris.target

# train set과 test set으로 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_df, iris_target, test_size=0.3, random_state=42)

# Decision Tree 분류기 객체 생성
dt = DecisionTreeClassifier(random_state=42)

# Random Search를 위한 매개변수 그리드 정의
param_grid = {'max_depth': [1,2,3,4,5], 'min_samples_split': [1, 2,3,4,5]}

# Random Search 수행
random_search = RandomizedSearchCV(dt, param_distributions=param_grid, cv=5)
random_search.fit(X_train, y_train)

# Random Search로부터 최적의 분류기 객체 추출
dt_best = random_search.best_estimator_

# 테스트 데이터에 대한 예측값 생성
predicted_labels = dt_best.predict(X_test)

# 오분류 테이블 생성
misclassification_table = pd.crosstab(y_test, predicted_labels)

# 가중치 평균 F1 스코어 계산
f1 = f1_score(y_test, predicted_labels, average='weighted')

# 정확도 계산
accuracy = accuracy_score(y_test, predicted_labels)

# 재현율 계산
recall = recall_score(y_test, predicted_labels, average='weighted')

# 정밀도 계산
precision = precision_score(y_test, predicted_labels, average='weighted')

# 오분류 테이블, F1 스코어, 정확도, 재현율, 정밀도 출력
misclassification_table, f1, accuracy, recall, precision

# 과적합 확인을 위한 학습 데이터에 대한 예측값 생성
train_predicted_labels = dt_best.predict(X_train)

# 학습 데이터에 대한 가중치 평균 F1 스코어 계산
train_f1 = f1_score(y_train, train_predicted_labels, average='weighted')

# 학습 데이터에 대한 정확도 계산
train_accuracy = accuracy_score(y_train, train_predicted_labels)

# 학습 데이터에 대한 재현율 계산
train_recall = recall_score(y_train, train_predicted_labels, average='weighted')

# 학습 데이터에 대한 정밀도 계산
train_precision = precision_score(y_train, train_predicted_labels, average='weighted')

# 학습 데이터에 대한 F1 스코어, 정확도, 오분류 테이블, 재현율, 정밀도 출력
train_f1, train_accuracy, misclassification_table, train_recall, train_precision, random_search.best_params_


c:\Users\de31\anaconda3\envs\lp_gpu\lib\site-packages\sklearn\model_selection\_validation.py:425: FitFailedWarning: 
10 fits failed out of a total of 50.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
10 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\de31\anaconda3\envs\lp_gpu\lib\site-packages\sklearn\model_selection\_validation.py", line 729, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\de31\anaconda3\envs\lp_gpu\lib\site-packages\sklearn\base.py", line 1145, in wrapper
    estimator._validate_params()
  File "c:\Users\de31\anaconda3\envs\lp_gpu\lib\site-packages\sklearn\base.py", line 638, in _validate_params
    validate_parameter_constraints(
  File 

(0.9523025599484354,
 0.9523809523809523,
 col_0   0   1   2
 row_0            
 0      19   0   0
 1       0  13   0
 2       0   0  13,
 0.9523809523809523,
 0.9543977591036416,
 {'min_samples_split': 2, 'max_depth': 3})

## 랜덤 포레스트

In [116]:
from sklearn.metrics import precision_score, recall_score

# 코드 실행을 위한 필요한 라이브러리 임포트
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split, RandomizedSearchCV

# Iris 데이터셋 로드
iris = load_iris()
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_target = iris.target

# 데이터를 학습 세트와 테스트 세트로 분할
X_train, X_test, y_train, y_test = train_test_split(iris_df, iris_target, test_size=0.3, random_state=42)

# 랜덤 포레스트 분류기 초기화
rf = RandomForestClassifier(random_state=42)

# 하이퍼파라미터 그리드 설정
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

# 랜덤 검색 객체 초기화
random_search = RandomizedSearchCV(estimator=rf, param_distributions=param_grid, cv=5)
random_search.fit(X_train, y_train)

# 최적의 파라미터와 모델 찾기
best_params = random_search.best_params_
best_rf = random_search.best_estimator_

# 테스트 데이터에 대한 예측 및 평가 지표 계산
predicted_labels = best_rf.predict(X_test)
misclassification_table = pd.crosstab(y_test, predicted_labels, rownames=['Actual'], colnames=['Predicted'])
f1_test = f1_score(y_test, predicted_labels, average='weighted')
accuracy_test = accuracy_score(y_test, predicted_labels)
precision_test = precision_score(y_test, predicted_labels, average='weighted')
recall_test = recall_score(y_test, predicted_labels, average='weighted')

# 학습 데이터에 대한 예측 및 평가 지표 계산
train_predicted_labels = best_rf.predict(X_train)
train_f1 = f1_score(y_train, train_predicted_labels, average='weighted')
train_accuracy = accuracy_score(y_train, train_predicted_labels)
precision_train = precision_score(y_train, train_predicted_labels, average='weighted')
recall_train = recall_score(y_train, train_predicted_labels, average='weighted')

# 결과 출력
(best_params, misclassification_table, f1_test, accuracy_test, precision_test, recall_test,
 train_f1, train_accuracy, precision_train, recall_train)


({'n_estimators': 100,
  'min_samples_split': 5,
  'min_samples_leaf': 1,
  'max_depth': None},
 Predicted   0   1   2
 Actual               
 0          19   0   0
 1           0  13   0
 2           0   0  13,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9809384572542468,
 0.9809523809523809,
 0.9819291819291819,
 0.9809523809523809)

## 그레디언트 부스팅

In [111]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

# iris 데이터셋 로드
iris = load_iris()

# iris 데이터프레임 생성
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# iris target 값 설정
iris_target = iris.target

# train 데이터와 test 데이터로 분할
X_train, X_test, y_train, y_test = train_test_split(iris_df, iris_target, test_size=0.3, random_state=42)

# GradientBoostingClassifier 모델 생성
gb = GradientBoostingClassifier(random_state=42)

# 탐색할 하이퍼파라미터 범위 설정
param_dist = {
    'n_estimators': randint(50, 201),
    'learning_rate': [0.1, 0.05, 0.01],
    'max_depth': [3, 4, 5]
}

# 랜덤 서치를 통한 최적의 하이퍼파라미터 탐색
random_search = RandomizedSearchCV(gb, param_distributions=param_dist, cv=5)
random_search.fit(X_train, y_train)

# 최적의 모델 선택
best_gb = random_search.best_estimator_

# 테스트 데이터로 예측
predicted_labels = best_gb.predict(X_test)

# 오분류 테이블 생성
misclassification_table = pd.crosstab(y_test, predicted_labels)

# F1 스코어 계산
f1 = f1_score(y_test, predicted_labels, average='weighted')

# 정확도 계산
accuracy = accuracy_score(y_test, predicted_labels)

# 오분류 테이블, F1 스코어, 정확도 출력
misclassification_table, f1, accuracy

# 과적합 확인을 위해 훈련 데이터로 예측
train_predicted_labels = best_gb.predict(X_train)

# 훈련 데이터의 F1 스코어 계산
train_f1 = f1_score(y_train, train_predicted_labels, average='weighted')

# 훈련 데이터의 정확도 계산
train_accuracy = accuracy_score(y_train, train_predicted_labels)

# 재현율 계산
recall = recall_score(y_test, predicted_labels, average='weighted')

# 정밀도 계산
precision = precision_score(y_test, predicted_labels, average='weighted')

# 훈련 데이터의 F1 스코어, 정확도,재현율, 정밀도, 오분류 테이블 출력
train_f1, train_accuracy, misclassification_table, recall, precision


(1.0,
 1.0,
 col_0   0   1   2
 row_0            
 0      19   0   0
 1       0  13   0
 2       0   0  13,
 1.0,
 1.0)

## 히스토그램 그레디언트 부스팅

In [114]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import f1_score, accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV

# iris 데이터셋 로드
iris = load_iris()

# iris 데이터를 DataFrame으로 변환
iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)

# iris target 값
iris_target = iris.target

# train set과 test set으로 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(iris_df, iris_target, test_size=0.3, random_state=42)

# HistGradientBoostingClassifier 모델 생성
gb = HistGradientBoostingClassifier(random_state=42)

# Hyperparameter 탐색을 위한 그리드 설정
param_grid = {
    'learning_rate': [0.01, 0.1, 0.5],
    'max_depth': [3, 5, 7],
    'max_leaf_nodes': [None, 5, 10],
    'min_samples_leaf': [1, 3, 5],
    'l2_regularization': [0.0, 0.1, 0.5]
}

# 랜덤 서치를 통한 Hyperparameter 탐색
random_search = RandomizedSearchCV(gb, param_distributions=param_grid, n_iter=10, cv=5, random_state=42)
random_search.fit(X_train, y_train)

# 최적의 모델 선택
gb_best = random_search.best_estimator_

# 테스트 데이터에 대한 예측
predicted_labels = gb_best.predict(X_test)

# 오분류 테이블 생성
misclassification_table = pd.crosstab(y_test, predicted_labels)

# F1 스코어 계산
f1 = f1_score(y_test, predicted_labels, average='weighted')

# 정확도 계산
accuracy = accuracy_score(y_test, predicted_labels)

# 오분류 테이블, F1 스코어, 정확도 출력
misclassification_table, f1, accuracy

# 과적합 확인을 위해 학습 데이터에 대한 예측
train_predicted_labels = gb_best.predict(X_train)

# 학습 데이터에 대한 F1 스코어 계산
train_f1 = f1_score(y_train, train_predicted_labels, average='weighted')

# 학습 데이터에 대한 정확도 계산
train_accuracy = accuracy_score(y_train, train_predicted_labels)

# 재현율과 정밀도 계산
from sklearn.metrics import recall_score, precision_score

# 테스트 데이터에 대한 재현율 계산
recall = recall_score(y_test, predicted_labels, average='weighted')

# 테스트 데이터에 대한 정밀도 계산
precision = precision_score(y_test, predicted_labels, average='weighted')


# 학습 데이터에 대한 F1 스코어, 정확도, 오분류 테이블 출력
train_f1, train_accuracy, misclassification_table, recall, precision

(1.0,
 1.0,
 col_0   0   1   2
 row_0            
 0      19   0   0
 1       0  11   2
 2       0   0  13,
 0.9555555555555556,
 0.9614814814814815)